# Guía 3 — Introducción a LangChain con OpenAI API
Curso: **IA en el Aula — Nivel Avanzado**  
Profesor: **Luis Daniel Benavides Navarro**  
Fecha: **Octubre 2025**

En esta guía aprenderás los conceptos básicos de **LangChain**, una librería diseñada para construir aplicaciones impulsadas por modelos de lenguaje, integrando la API de OpenAI con flujos más complejos. Exploraremos cómo crear un primer **prompt chain**, administrar memoria y usar herramientas para componer respuestas inteligentes.

## 1️⃣ Instalación y configuración inicial
LangChain se instala como cualquier otra librería de Python. También usaremos `python-dotenv` para gestionar la clave de OpenAI. Ejecute la siguiente celda:

In [3]:
%pip install openai langchain python-dotenv langchain-openai

Defaulting to user installation because normal site-packages is not writeable
  Using cached openai-2.6.1-py3-none-any.whl.metadata (29 kB)
  Using cached langchain-1.0.2-py3-none-any.whl.metadata (4.7 kB)
  Using cached langchain_openai-1.0.1-py3-none-any.whl.metadata (1.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.11.1-cp311-cp311-win_amd64.whl.metadata (5.3 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached langchain_core-1.0.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached langgraph-1.0.1-py3-none-any.whl.metadata (7.4 kB)
  Using cached pydantic-2.12.3-py3-none-any.whl.metadata (87 kB)
  Using cached tiktoken-0.12.0-cp311-cp311-win_amd64.whl.metadata (6.9 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached langsmith-0.4.38-py3-none

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-cloud-auth 0.1.4 requires pydantic<2.0, but you have pydantic 2.12.3 which is incompatible.
streamlit 1.30.0 requires packaging<24,>=16.8, but you have packaging 25.0 which is incompatible.


## 2️⃣ Cargar variables de entorno y cliente OpenAI
Crea un archivo `.env` con tu clave de API y cárgala para poder usarla dentro de LangChain. Esto evita exponer tu clave en el código fuente.

In [1]:

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()  # Cargar archivo .env
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("No se encontró la clave OPENAI_API_KEY en el archivo .env")

# Crear cliente LangChain con OpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
print("Cliente LangChain con OpenAI inicializado correctamente.")


c:\Users\geronimo.martinez-n\Downloads\taller de LLM\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cliente LangChain con OpenAI inicializado correctamente.


## 3️⃣ Primer ejemplo: Prompt simple con LangChain
LangChain utiliza **chains** (cadenas de pasos) para procesar información. Comencemos con una cadena simple que envía un mensaje al modelo y obtiene la respuesta.

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize the LLM (uses your OpenAI API key from environment)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)

# Create a simple prompt
prompt = ChatPromptTemplate.from_template(
    "Explica en dos frases el concepto de {tema}."
)

# Combine the components using LCEL (LangChain Expression Language)
chain = prompt | llm | StrOutputParser()

# Run it
result = chain.invoke({"tema": "aprendizaje automático"})
print(result)


El aprendizaje automático es una rama de la inteligencia artificial que permite a las computadoras aprender de datos y mejorar su rendimiento en tareas específicas sin ser programadas explícitamente para ello. Utiliza algoritmos para identificar patrones y hacer predicciones o decisiones basadas en la información que ha procesado.


### Explicación
- **ChatPromptTemplate:** define la estructura del mensaje que se envía al modelo.
- **ChatOpenAI:** la conexión real al modelo.
- **chain** crea la cadena de componentes usando LCEL (LangChain Expression Language).
- **StrOutputParser** convierte la salida estructurada del modelo en texto plano.
- **chain.invoke:** ejecuta la cadena pasando los valores del prompt.

Este patrón permite reutilizar prompts para distintos temas o contextos.

## 4️⃣ Ejemplo 2: Encadenar múltiples pasos
LangChain permite combinar varios pasos en una misma ejecución. En este ejemplo, crearemos dos prompts: uno para definir un tema y otro para generar una aplicación educativa basada en el resultado.

In [2]:
# --- Importaciones modernas ---
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI  # o tu modelo preferido
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# --- Definir el modelo LLM ---
# Puedes reemplazar el modelo si usas otro (por ejemplo, de Hugging Face o local)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

# --- Primer paso: obtener una descripción ---
primer_prompt = PromptTemplate(
    input_variables=["tema"],
    template="Explica brevemente el concepto de {tema}."
)

# --- Segundo paso: generar aplicación educativa ---
segundo_prompt = PromptTemplate(
    input_variables=["concepto"],
    template="Propón una aplicación educativa del siguiente concepto: {concepto}."
)

# --- Construcción de la cadena secuencial ---
# Paso 1: toma 'tema' → genera 'concepto'
primer_chain = primer_prompt | llm | StrOutputParser()

# Paso 2: toma el resultado del paso anterior ('concepto') → genera aplicación
segundo_chain = segundo_prompt | llm | StrOutputParser()

# --- Encadenar ambos pasos ---
chain_secuencial = (
    {"tema": RunnablePassthrough()}  # pasa el input "realidad aumentada"
    | primer_chain
    | (lambda concepto: {"concepto": concepto})  # renombra la salida para el siguiente prompt
    | segundo_chain
)

# --- Ejecutar la cadena completa ---
resultado = chain_secuencial.invoke("realidad aumentada")
print(resultado)


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: oaRbaUvz************************************************************************************************************************************************JoQA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

# 1) LLM (usa tu OPENAI_API_KEY en el entorno)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
to_str = StrOutputParser()

# 2) Paso 1: explicar brevemente el concepto de {tema}
primer_prompt = ChatPromptTemplate.from_template(
    "Explica brevemente el concepto de {tema}."
)
primer_paso = primer_prompt | llm | to_str
# `primer_paso` produce un string, por ejemplo: "La realidad aumentada es ..."

# 3) Paso 2: proponer una aplicación educativa usando la salida del paso 1 como {concepto}
segundo_prompt = ChatPromptTemplate.from_template(
    "Propón una aplicación educativa del siguiente concepto: {concepto}."
)
segundo_paso = segundo_prompt | llm | to_str

# 4) Encadenar: mapear la entrada {tema} al primer paso, y su salida a {concepto} del segundo
cadena_secuencial = {"concepto": primer_paso} | segundo_paso

# 5) Ejecutar la cadena completa
resultado = cadena_secuencial.invoke({"tema": "realidad aumentada"})
print(resultado)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************n6gA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

### Explicación
- **cadena_secuencial:** permite conectar varias cadenas; la salida de una se convierte en la entrada de la siguiente.
- En este caso, el modelo primero explica el tema y luego sugiere una aplicación educativa.

Este tipo de flujo es ideal para generar contenido didáctico o ideas para proyectos en clase.

## 5️⃣ Ejemplo 3: Añadir memoria a la conversación
LangChain incluye módulos de **memoria** para mantener contexto entre múltiples interacciones. Esto permite simular conversaciones educativas más naturales, donde el modelo recuerda temas previos.

In [10]:

# Instalar si hace falta:
# %pip install -U langchain langchain-openai

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser

# LLM (requiere OPENAI_API_KEY en el entorno)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
to_str = StrOutputParser()

# Prompt con hueco para el historial
prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente educativo claro y conciso."),
    MessagesPlaceholder("chat_history"),      # ← aquí va la memoria
    ("human", "{input}")
])

# Cadena base
chain = prompt | llm | to_str

# Memoria simple como lista de mensajes
history: list = []

def chat(user_text: str) -> str:
    """
    Envía un turno del usuario, usa el historial y actualiza la memoria
    con el par (usuario, asistente).
    """
    global history
    # Ejecutar la cadena inyectando el historial actual
    answer = chain.invoke({"input": user_text, "chat_history": history})
    # Actualizar memoria (guardar los dos mensajes)
    history += [HumanMessage(content=user_text), AIMessage(content=answer)]
    return answer

# --- Ejemplo de uso (tres turnos) ---
print(chat("Hola, soy un profesor de informática."))
print(chat("¿Puedes explicarme cómo introducir IA a mis estudiantes?"))
print(chat("¿Qué ejemplos prácticos puedo usar en la clase?"))



AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************n6gA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

### Explicación
- **ConversationBufferMemory:** almacena los mensajes anteriores en la conversación.
- **ConversationChain:** combina el modelo con la memoria.
- Esta funcionalidad es útil para tutores inteligentes o chatbots educativos que requieren continuidad en el diálogo.

## 6️⃣ Buenas prácticas con LangChain + OpenAI
- Usa prompts **claros y estructurados**: el modelo responde mejor cuando la tarea está bien definida.
- Controla `temperature` según la tarea: bajo (0.1–0.3) para precisión, alto (0.7–0.9) para creatividad.
- Guarda logs o historiales si tu aplicación incluye interacción prolongada.
- Limita la longitud de las respuestas con `max_tokens` para evitar costos o respuestas excesivas.
- Documenta tus cadenas (`LLMChain`) para reusarlas en distintos contextos educativos.

## ✅ Conclusión
Has aprendido los fundamentos de LangChain: prompts, chains, memoria y flujo secuencial. Estos conceptos son la base para desarrollar asistentes educativos, tutores personalizados o sistemas de generación de contenido en el aula. En la próxima guía implementaremos un **asistente educativo con RAG (Retrieval-Augmented Generation)** usando tus propios materiales docentes.